# Classifying Pulsar Candidates Using a K-NN Algorithm

## Introduction
- Pulsars are rotating neutron stars observed to have pulses of radiation at very regular intervals, typically ranging from milliseconds to seconds. These accelerated particles produce very powerful beams of light, with some pulsars producing enough radio emissions to be detected on Earth (Ng et al., 2015).
- Detection of Pulsars is a complex task and involves discerning fleeting Pulsar signals from a copious amount of background radio frequency interference (Eatough et al., 2010). To aid in the detection of Pulsars, computer algorithms can speed up the process and accuracy. Within our project, we aim to create a supervised *K*-Nearest Neighbors algorithm in the binary classification of the Pulsar star state.



**Predictive question:** Can we create a *K*-nearest neighbor algorithm capable of classifying whether an observation is a Pulsar star or not, given the kurtosis and skewness of the integrated profile, as well as the mean, kurtosis, and skewness of the DM-SNR curve?


#### Dataset
- The data set that we use describes a sample of pulsar candidates collected as part of the High Time Resolution Universe Survey of the southern hemisphere.
- The data set contains a total of 8 different features and 1 class variable. The variables are measures of the dispersion measure-signal-to-noise ratio (DM-SNR) or the integrated profile of each recorded observation.

## Exploratory data analysis

In [1]:
# Importing libraries and setting seed
library(tidyverse)
library(tidymodels)

set.seed(100)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

#### Reading and wrangling the data from internet:

In [2]:
# Since the initial data file is in a zip folder, we created a new repository on github, and 
# utilized the raw link to demonstrate that the data can be read from the web

pulsar_df <- read_csv("https://raw.githubusercontent.com/MichaelZhang33/HTRU2.csv/main/HTRU_2.arff", 
                      skip = 11, col_names = FALSE) |>
    rename("Profile_mean" = X1,
       "Profile_stdev" = X2,
       "Profile_skewness" = X3,
       "Profile_kurtosis" = X4,
       "DM_mean" = X5, 
       "DM_stdev" = X6, 
       "DM_skewness" = X7,
       "DM_kurtosis" = X8,
       "class" = X9) |>

# Further wrangling and cleaning the data
    mutate(class = as_factor(class)) |>
    mutate(Class = fct_recode(class, "notpulsar" = "0", "pulsar" = "1")) |>
    select(-Profile_mean, -Profile_stdev, -DM_stdev, -class)

head(pulsar_df)
 

# Splitting the dataset into a training and testing set
pulsar_split <- initial_split(pulsar_df, prop = 0.75, strata = Class)
pulsar_train <- training(pulsar_split)
pulsar_test <- testing(pulsar_split)

ERROR: Error in open.connection(structure(4L, class = c("curl", "connection"), conn_id = <pointer: 0x380>), : HTTP error 404.


### Useful Tabular Data

##### 1. Summary table for the number of pulsar star observations and non-pulsar star observations in the training set:

In [ ]:
useful_df_1 <- pulsar_train |>
    group_by(Class) |>
    summarize(count = n())
useful_df_1

# These results indicate that the dataset is not balanced, and will require oversampling of the positive 
# pulsar cases when creating the model.

##### 2. Summary table for the average value of each predictor variable based on class:

In [ ]:
useful_df_2 <- pulsar_train |>
    group_by(Class) |>
    summarize(mean_profile_skewness = mean(Profile_skewness), 
              mean_profile_kurtosis = mean(Profile_kurtosis), 
              mean_DM_mean = mean(DM_mean), 
              mean_DM_skewness = mean(DM_skewness), 
              mean_DM_kurtosis = mean(DM_kurtosis))
useful_df_2

##### 3. Number of rows in each column that have na values:

In [ ]:
useful_df_3 <- colSums(is.na(pulsar_train))
useful_df_3

### Useful visualisations

#### 1. Distribution of Profile_skewness

In [ ]:
profile_sk_distribution <- pulsar_train |>
    select(Profile_skewness, Class) |>
    ggplot(aes(x = Profile_skewness)) +
    geom_histogram(bins = 10) +
    labs(x= "Skewness of the integrated profile") +
    ggtitle("Distribution of the skewness of the integrated profile") +
    theme(text = element_text(size = 15)) +
    facet_grid(cols = vars(Class))
profile_sk_distribution

#### 2. Distribution of Profile_kurtosis

In [ ]:
profile_kur_distribution <- pulsar_train |>
    select(Profile_kurtosis, Class) |>
    ggplot(aes(x = Profile_kurtosis)) +
    geom_histogram(bins = 10) +
    labs(x= "Excess kurtosis of integrated profile") +
    ggtitle("Distribution of the excess kurtosis of integrated profile") +
    theme(text = element_text(size = 15)) +
    facet_grid(cols = vars(Class))
profile_kur_distribution

#### 3. Distribution of DM_mean

In [ ]:
profile_dmMean_distribution <- pulsar_train |>
    select(DM_mean, Class) |>
    ggplot(aes(x = DM_mean)) +
    geom_histogram(bins = 10) +
    labs(x= "Mean of DM-SNR curve") +
    ggtitle("Distribution of the mean of DM-SNR curve") +
    theme(text = element_text(size = 15)) +
    facet_grid(cols = vars(Class))
profile_dmMean_distribution

#### 4. Distribution of DM_skewness

In [ ]:
profile_dmsk_distribution <- pulsar_train |>
    select(DM_skewness, Class) |>
    ggplot(aes(x = DM_skewness)) +
    geom_histogram(bins = 10) +
    labs(x= "Skewness of DM-SNR curve") +
    ggtitle("Distribution of the skewness of DM-SNR curve") +
    theme(text = element_text(size = 15)) +
    facet_grid(cols = vars(Class))
profile_dmsk_distribution

#### 5. Distribution of DM_kurtosis

In [ ]:
profile_dmKur_distribution <- pulsar_train |>
    select(DM_kurtosis, Class) |>
    ggplot(aes(x = DM_kurtosis)) +
    geom_histogram(bins = 10) +
    labs(x= "Kurtosis of DM-SNR curve") +
    ggtitle("Distribution of the kurtosis of DM-SNR curve") +
    theme(text = element_text(size = 15)) +
    facet_grid(cols = vars(Class))
profile_dmKur_distribution

## Methods
To answer our question, we will utilize some of the columns from the initial dataset. These include:
 
1. The excess kurtosis of the integrated profile (*Profile_kurtosis*)
2. The skewness of the integrated profile (*Profile_skewness*)
3. Mean of the DM-SNR curve (*DM_mean*)
4. Excess kurtosis of the DM-SNR curve (*DM_kurtosis*)
5. Skewness of the DM-SNR curve (*DM_skewness*)
6. Observation Class (the outcome variable)

Using the first 5 columns, we will predict through *classification* to determine the observation class.
These variables were chosen based on the nature of Pulsar stars as powerful yet fleeting signals, where the kurtosis and skewness of the integrated profile will help distinguish the transient nature of the star. Furthermore, the DM-SNR mean, kurtosis and skewness will indicate the strength and nature of the signal in comparison to background noise and radio frequency interference.


**Visualisation:** We will visualize our results by creating a confusion matrix to visualize the performance of the model. Furthermore, we will utilize scatterplots to visualize the relationship between the predictor variables. 

## Expected outcomes and significance

**What we expect to find**
- For Pulsar stars, we expect a higher integrated profile DM_mean value and smaller kurtosis to be correlated with Pulsar stars, as it would indicate a more intense, fleeting beam of light. Furthermore, we expect there to be a lower skewness, and 
We expect to find correlations between a higher profile kurtosis and skewness, and a 

**Impacts of our finding**
- This data analysis can help scientists in classifying if an observation is a Pulsar star, a matter of great scientific interest due to the nature of the stars as gravitational probes.

**Future questions this data analysis could lead to:** 
- Are there any other factors that can help in determining whether an observation is a Pulsar star?
- Are there other algorithms that may be more effective in classifying Pulsar status compared to K-NN? 

#### <center> References </center>
Eatough, R. P., Molkenthin, N., Kramer, M., Noutsos, A., Keith, M. J., Stappers, B. W., & Lyne, A. G. (2010). Selection of radio pulsar candidates using artificial neural networks. Monthly Notices of the Royal Astronomical Society, 407(4), 2443–2450. https://doi.org/10.1111/j.1365-2966.2010.17082.x


Ng, C., Champion, D. J., Bailes, M., Barr, E. D., Bates, S. D., Bhat, N. D. R., Burgay, M., Burke-Spolaor, S., Flynn, C. M. L., Jameson, A., Johnston, S., Keith, M. J., Kramer, M., Levin, L., Petroff, E., Possenti, A., Stappers, B. W., van Straten, W., Tiburzi, C., & Eatough, R. P. (2015). The High Time Resolution Universe Pulsar Survey – XII. Galactic plane acceleration search and the discovery of 60 pulsars. Monthly Notices of the Royal Astronomical Society, 450(3), 2922–2947. https://doi.org/10.1093/mnras/stv753

